<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-de-pacotes" data-toc-modified-id="Import-de-pacotes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import de pacotes</a></span></li><li><span><a href="#Leitura-do-dado-tratado" data-toc-modified-id="Leitura-do-dado-tratado-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Leitura do dado tratado</a></span></li><li><span><a href="#Modelo-naive" data-toc-modified-id="Modelo-naive-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modelo naive</a></span><ul class="toc-item"><li><span><a href="#Exemplo" data-toc-modified-id="Exemplo-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Exemplo</a></span></li><li><span><a href="#Exemplo-iterativo-em-t" data-toc-modified-id="Exemplo-iterativo-em-t-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Exemplo iterativo em t</a></span></li></ul></li></ul></div>

## Import de pacotes

In [1]:
import os,sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from math import factorial

from scipy import stats

import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

sys.path.append('../..')
from config import *
from utils import *

## Leitura do dado tratado

In [3]:
df = pd.read_pickle(os.path.join(TRTD_DATA_PATH,'eventos.pkl'))

## Modelo naive

In [4]:
def likelihood(d,mu):
    return sum([mu - y*np.log(mu) for y in d])

class NaiveModel:
    def __init__(self):
        self.lambda_ = None
        self.dist = None
        self.likelihood = None
    
    def fit(self,s):
        self.dist = s
        self.lambda_ = s.mean()
        
        self.likelihood = likelihood(self.dist,self.lambda_)
        
    def pdf(self):
        return np.array([stats.poisson(self.lambda_).pmf(x) for x in range(1,4)])

### Exemplo

In [ ]:
model = NaiveModel()
d = calcular_distribuicao(df,29,1,29,1)
model.fit(d)
model.likelihood

In [ ]:
# estimador de maxima verossimilhanca
lambda_ = d.mean()
# distribuicao real
d = pd.Series(d).value_counts() / len(d)
# distribuicao teorica
teor_d = np.array([stats.poisson(lambda_).pmf(x) for x in range(1,10)])

print(lambda_)
plt.bar(x=d.index,height=d.values)
plt.plot(teor_d,color='red')
plt.show()

### Exemplo iterativo em t

In [ ]:
for it in range(1,49):
    model = NaiveModel()
    d = calcular_distribuicao(df,it,1,29,1)
    model.fit(d)
    print('t: {} - Likelihood: {}'.format(it,model.likelihood))